In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions.{datediff, hour, to_date, udf,lower,concat,lit, from_unixtime,format_number,when}

Intitializing Scala interpreter ...

Spark Web UI available at http://10.78.180.192:4040
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1572258697359)
SparkSession available as 'spark'


import org.apache.spark.SparkConf
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions.{datediff, hour, to_date, udf, lower, concat, lit, from_unixtime, format_number, when}


In [2]:
val df : DataFrame = spark.read
      .option("header",true)
      .option("inferSchema","true")
      .csv("data/train_clean.csv")

df: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 12 more fields]
